### Training process

- Design model (input size, output size, forward pass)
- Construct loss and optimizer
- Training loop
  - forward pass: compute prediction
  - backward pass: gradients
  - update weights


### Import package


In [12]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Prepared data


In [13]:
bc = datasets.load_breast_cancer()
# x: (569sample, 30feature)
# y: classification target
x, y = bc.data, bc.target

n_samples, n_features = x.shape
print(n_samples, n_features)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=123)

# scale 因為資料分布太大，用StandardScaler將資料標準化，平均值為0，標準差為1
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
# 若在測試資料上使用fit_transform，則計算的是測試數據自身的平均值和標準差，這可能導致兩組數據處理過程不一致
# 而transform是在已知這些統計數據的情況下，將同樣的轉換應用於測試資料，以保持一致的前處理方式。
x_test = sc.transform(x_test)

# numpy->torch
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

569 30


### Model


In [14]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred


model = LogisticRegression(n_features)

### Loss and optimizer


In [15]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training loop


In [16]:
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)

    # backward
    loss.backward()

    # update
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

epoch: 10, loss: 0.5412
epoch: 20, loss: 0.4536
epoch: 30, loss: 0.3979
epoch: 40, loss: 0.3589
epoch: 50, loss: 0.3298
epoch: 60, loss: 0.3071
epoch: 70, loss: 0.2888
epoch: 80, loss: 0.2736
epoch: 90, loss: 0.2607
epoch: 100, loss: 0.2496


### Evaluate model


In [17]:
# 評估模型時不需要梯度
with torch.no_grad():
    y_pred = model(x_test)
    # 四捨五入
    y_pred_cls = y_pred.round()
    # y_pred_cls.eq(y_test) 會產生一個布林張量，表示每個預測是否正確
    acc = y_pred_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9825
